# Dataframe tutorial - Advanced
In this tutorial we will cover
- GroupBy and agreegate functions
- Data joins
- Movies recommendation using MLLib

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('dataframe-advanced').getOrCreate()

spark

24/02/13 15:44:26 WARN Utils: Your hostname, ndi-mb-275 resolves to a loopback address: 127.0.0.1; using 192.168.29.185 instead (on interface en0)
24/02/13 15:44:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/13 15:44:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
ratings_df = spark.read.option("delimiter", "::").csv("../sample_data/movie-lense/ratings.dat", inferSchema=True, header=True)
ratings_df.show()

+------+-------+------+---------+
|UserID|MovieID|Rating|Timestamp|
+------+-------+------+---------+
|     1|   1193|     5|978300760|
|     1|    661|     3|978302109|
|     1|    914|     3|978301968|
|     1|   3408|     4|978300275|
|     1|   2355|     5|978824291|
|     1|   1197|     3|978302268|
|     1|   1287|     5|978302039|
|     1|   2804|     5|978300719|
|     1|    594|     4|978302268|
|     1|    919|     4|978301368|
|     1|    595|     5|978824268|
|     1|    938|     4|978301752|
|     1|   2398|     4|978302281|
|     1|   2918|     4|978302124|
|     1|   1035|     5|978301753|
|     1|   2791|     4|978302188|
|     1|   2687|     3|978824268|
|     1|   2018|     4|978301777|
|     1|   3105|     5|978301713|
|     1|   2797|     4|978302039|
+------+-------+------+---------+
only showing top 20 rows



In [9]:
ratings_df.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- MovieID: integer (nullable = true)
 |-- Rating: integer (nullable = true)
 |-- Timestamp: integer (nullable = true)



In [3]:
# count rating submitted by each user
from pyspark.sql.functions import desc
ratings_df.groupBy('userid').count().sort(desc('count')).show()

+------+-----+
|userid|count|
+------+-----+
|  4169| 2314|
|  1680| 1850|
|  4277| 1743|
|  1941| 1595|
|  1181| 1521|
|   889| 1518|
|  3618| 1344|
|  2063| 1323|
|  1150| 1302|
|  1015| 1286|
|  5795| 1277|
|  4344| 1271|
|  1980| 1260|
|  2909| 1258|
|  1449| 1243|
|  4510| 1240|
|   424| 1226|
|  4227| 1222|
|  5831| 1220|
|  3841| 1216|
+------+-----+
only showing top 20 rows



In [4]:
# get each user's ratings average
import pyspark.sql.functions as F
ratings_df.groupBy('userid').agg(F.avg('rating').alias('avg_rating')).show()

+------+------------------+
|userid|        avg_rating|
+------+------------------+
|   148| 3.733974358974359|
|   463|               3.0|
|   471|3.6285714285714286|
|   496| 4.294117647058823|
|   833|4.0476190476190474|
|  1088|3.3375850340136055|
|   243| 3.606060606060606|
|   392|3.4743326488706368|
|   540| 3.230769230769231|
|   623| 4.226744186046512|
|   737|3.5668202764976957|
|   858| 4.189473684210526|
|   897|3.5566502463054186|
|  1025| 3.757575757575758|
|  1084|              3.15|
|    31|  3.73109243697479|
|   516| 2.931972789115646|
|    85|3.1025641025641026|
|   137| 3.611940298507463|
|   251|3.5753424657534247|
+------+------------------+
only showing top 20 rows



In [8]:
# find users who have not given any 5 star or 4 star rating
import pyspark.sql.functions as F
ratings_df.groupBy('userid').agg(F.max('rating').alias('max_rating')).filter('max_rating<4').show()

+------+----------+
|userid|max_rating|
+------+----------+
|  3598|         2|
|  4486|         3|
+------+----------+



In [55]:
# find movies that are only rated 5 star and by more than 1 user
ratings_df.groupBy('movieid').agg(F.min('rating'), F.count('userid')).filter('min(rating)>4 and count(userid)>1').show()

+-------+-----------+-------------+
|movieid|min(rating)|count(userid)|
+-------+-----------+-------------+
|    787|          5|            3|
|   3233|          5|            2|
+-------+-----------+-------------+



In [5]:
# load users
users_df = spark.read.option("delimiter", "::").csv("../sample_data/movie-lense/users.dat", inferSchema=True, header=True)
users_df.show()

+------+------+---+----------+--------+
|UserID|Gender|Age|Occupation|Zip-code|
+------+------+---+----------+--------+
|     1|     F|  1|        10|   48067|
|     2|     M| 56|        16|   70072|
|     3|     M| 25|        15|   55117|
|     4|     M| 45|         7|   02460|
|     5|     M| 25|        20|   55455|
|     6|     F| 50|         9|   55117|
|     7|     M| 35|         1|   06810|
|     8|     M| 25|        12|   11413|
|     9|     M| 25|        17|   61614|
|    10|     F| 35|         1|   95370|
|    11|     F| 25|         1|   04093|
|    12|     M| 25|        12|   32793|
|    13|     M| 45|         1|   93304|
|    14|     M| 35|         0|   60126|
|    15|     M| 25|         7|   22903|
|    16|     F| 35|         0|   20670|
|    17|     M| 50|         1|   95350|
|    18|     F| 18|         3|   95825|
|    19|     M|  1|        10|   48073|
|    20|     M| 25|        14|   55113|
+------+------+---+----------+--------+
only showing top 20 rows



In [31]:
users_df.printSchema()

root
 |-- UserID: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Occupation: integer (nullable = true)
 |-- Zip-code: string (nullable = true)



In [6]:
# load movies
movies_df = spark.read.option("delimiter", "::").csv("../sample_data/movie-lense/movies.dat", inferSchema=True, header=True)
movies_df.show()

+-------+--------------------+--------------------+
|MovieID|               Title|              Genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Animation|Childre...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|        Comedy|Drama|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|Adventure|Children's|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Animation|Children's|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|      Drama|Thriller|
|     17|Sen

In [57]:
movies_df.printSchema()

root
 |-- MovieID: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genres: string (nullable = true)



In [60]:
# show movie names that only rated 5 star
df1 = ratings_df.groupBy('movieid').agg(F.min('rating'), F.count('userid')).filter('min(rating)>4')
movies_df.join(df1, 'movieid').show()

+-------+--------------------+--------------------+-----------+-------------+
|MovieID|               Title|              Genres|min(rating)|count(userid)|
+-------+--------------------+--------------------+-----------+-------------+
|    787|Gate of Heavenly ...|         Documentary|          5|            3|
|   3280|    Baby, The (1973)|              Horror|          5|            1|
|    989|Schlafes Bruder (...|               Drama|          5|            1|
|   1830|Follow the Bitch ...|              Comedy|          5|            1|
|   3881|Bittersweet Motel...|         Documentary|          5|            1|
|   3172|Ulysses (Ulisse) ...|           Adventure|          5|            1|
|   3233|Smashing Time (1967)|              Comedy|          5|            2|
|   3607|One Little Indian...|Comedy|Drama|Western|          5|            1|
|   3382|Song of Freedom (...|               Drama|          5|            1|
|   3656|        Lured (1947)|               Crime|          5| 

In [16]:
# find the movies with lowest average rating and rated by at least 5 users
df1 = ratings_df.groupBy('movieid').agg(F.avg('rating'), F.count('userid')).filter('count(userid)>4')
movies_df.join(df1, 'movieid').sort(F.asc('avg(rating)')).show()

+-------+--------------------+--------------------+------------------+-------------+
|MovieID|               Title|              Genres|       avg(rating)|count(userid)|
+-------+--------------------+--------------------+------------------+-------------+
|   1311|Santa with Muscle...|              Comedy|               1.0|            7|
|   3574|Carnosaur 3: Prim...|       Horror|Sci-Fi|1.0588235294117647|           17|
|   3027|Slaughterhouse 2 ...|              Horror|1.1428571428571428|            7|
|   3166| Brenda Starr (1989)|           Adventure|1.1666666666666667|            6|
|   1720| Time Tracers (1995)|Action|Adventure|...|1.2857142857142858|            7|
|   1495|Turbo: A Power Ra...|Action|Adventure|...|1.3181818181818181|           44|
|   1739|3 Ninjas: High No...|   Action|Children's|1.3617021276595744|           47|
|   1323|Amityville 3-D (1...|              Horror| 1.372093023255814|           43|
|   3573|  Carnosaur 2 (1995)|       Horror|Sci-Fi|1.461538461538

# Find movie recommendation for a user

In [7]:
movies_names = { row['MovieID']: row['Title'] for row in movies_df.rdd.collect() }
movies_names[1]

'Toy Story (1995)'

In [8]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [9]:
(training_df, test_df) = ratings_df.randomSplit([0.9, 0.1], seed=10)
training_df.count()

900378

In [10]:
als = ALS(maxIter=20, regParam=0.0001, userCol='UserID', itemCol='MovieID', ratingCol='Rating')
model = als.fit(training_df)

24/02/13 15:45:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/02/13 15:45:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [11]:
 predictions = model.transform(test_df)

In [12]:
predictions.select('prediction').filter("prediction == 'NaN'").show()

+----------+
|prediction|
+----------+
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
|       NaN|
+----------+



In [99]:
type(predictions.na.drop())

pyspark.sql.dataframe.DataFrame

In [14]:
predictions.show()

+------+-------+------+---------+----------+
|UserID|MovieID|Rating|Timestamp|prediction|
+------+-------+------+---------+----------+
|     2|    110|     5|978298625| 4.3319182|
|     1|    260|     4|978300760|  4.262231|
|     2|    434|     2|978300174| 2.9618578|
|     2|    457|     4|978299773| 4.2564163|
|     2|    498|     3|978299418| 2.7943366|
|     3|    733|     5|978297757|   3.06048|
|     2|    902|     2|978298905|  4.040274|
|     1|    938|     4|978301752| 3.6273332|
|     3|   1196|     4|978297539| 4.7622943|
|     3|   1431|     3|978297095|   3.30723|
|     2|   1537|     4|978299620| 3.8317578|
|     2|   1552|     3|978299941| 3.0869236|
|     1|   1907|     4|978824330| 4.1323614|
|     3|   1968|     4|978297068|  4.425769|
|     1|   2018|     4|978301777| 4.8495274|
|     3|   2081|     4|978298504|  3.938663|
|     1|   2321|     3|978302205| 3.4895494|
|     1|   2340|     3|978300103|  5.087006|
|     2|   2396|     4|978299641| 4.2986975|
|     2|  

In [13]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='Rating', predictionCol='prediction')
rmse = evaluator.evaluate(predictions.na.drop(subset=['prediction']))
print(rmse)

0.9854597509750324


In [23]:
# predicting for specific user
user_1 = ratings_df.filter('UserId==100')
rec = model.transform(user_1)
rec.orderBy('prediction', ascending=False).show()

+------+-------+------+---------+----------+
|UserID|MovieID|Rating|Timestamp|prediction|
+------+-------+------+---------+----------+
|   100|    919|     5|977594947|  4.252399|
|   100|    527|     5|977594839| 3.9516923|
|   100|    969|     4|977594044| 3.9234064|
|   100|   1198|     4|977593607| 3.8870273|
|   100|    260|     4|977593595| 3.8632584|
|   100|    608|     3|977594932| 3.7646499|
|   100|   1197|     4|977594079| 3.7377713|
|   100|    858|     4|977593950|  3.726382|
|   100|    318|     4|977594839| 3.7216098|
|   100|   1387|     3|977594008|  3.680271|
|   100|    593|     4|977594809| 3.6374977|
|   100|   1196|     4|977593950|  3.614536|
|   100|   1304|     3|977593988| 3.5550394|
|   100|   2406|     4|977594142| 3.5022452|
|   100|    924|     4|977594873| 3.4842052|
|   100|    800|     5|977593915| 3.4827921|
|   100|    541|     3|977593624|  3.481179|
|   100|   2028|     4|977593988|  3.460435|
|   100|   1200|     3|977594021| 3.4595747|
|   100|  